# Оценка качества Retrieval: Множественные метрики

Этот ноутбук выполняет комплексную оценку качества поискового движка:
- Загружает запросы из `queries.json` (по 5 синтетических запросов на каждый пост)
- Выполняет поиск топ-10 документов для каждого запроса
- Считает метрики для K = 1, 3, 5, 10

## Метрики
1. **Hit Rate@K (Recall@K)** - доля запросов, где якорный документ попал в топ-K
2. **MRR@K** (Mean Reciprocal Rank) - среднее значение обратного ранга первого релевантного документа
3. **Pool-normalized Recall@K** - доля релевантных документов в top-K, нормированная на min(релевантных в пуле, K)

## Методология
- **Ground truth**: для каждого запроса известен якорный пост (из `queries.json`)
- Получаем топ-10 результатов от системы поиска
- Считаем все метрики одновременно для K ∈ {1, 3, 5, 10}

## Выходные файлы
- `report.txt` - полный текстовый отчёт со всеми логами и результатами
- `eval_results_multi_k.json` - детальные результаты в JSON формате

## 1. Импорты и настройки

In [ ]:
import json
from tplexity.tg_parse.chunker import PostChunker

# 1. Загрузить JSON
with open("./eval_data/messages_diverse_1000posts_all_channels.json", "r", encoding="utf-8") as f:
    posts = json.load(f)  # Может быть список или один словарь

# 2. Создать чанкер
chunker = PostChunker(
    source_name="./eval_data/messages_diverse_1000posts_all_channels.json",
    chunk_size=1000,
    chunk_overlap=100
)

# 3. Обработать посты и собрать чанки с сохранением всех полей
all_chunks = []
for post in posts:
    chunks = chunker.chunk_post(post)
    
    # Для каждого чанка создаем объект со всеми полями исходного поста
    for chunk_data in chunks:
        chunk_obj = {
            # Все поля из исходного поста
            "id": post.get("id"),
            "channel_id": post.get("channel_id"),
            "link": post.get("link"),
            "date": post.get("date"),
            "views": post.get("views"),
            "forwards": post.get("forwards"),
            "edit_date": post.get("edit_date"),
            "has_media": post.get("has_media"),
            "media_type": post.get("media_type"),
            "chunk_index": chunk_data["chunk_index"],
            "text": chunk_data["chunk_text"],  # chunk_text -> text
        }
        all_chunks.append(chunk_obj)

# 4. Сохранить в JSON
with open("chunks_output.json", "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)

print(f"Сохранено {len(all_chunks)} чанков")

/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Сохранено 1702 чанков


In [2]:
import asyncio
import json
import logging
import sys
from pathlib import Path    
from typing import Dict, List
from datetime import datetime

# Добавляем путь к проекту
project_root = Path("/srv/nlp1/eval_dir/T-bank_NLP_")
sys.path.insert(0, str(project_root))

# Настройка логирования в файл и консоль
report_path = project_root / "src/eval/report.txt"

# Класс для дублирования вывода в файл и консоль
class DualLogger:
    def __init__(self, filepath):
        self.terminal = sys.stdout
        self.log = open(filepath, "w", encoding="utf-8")
        
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        self.log.flush()
        
    def flush(self):
        self.terminal.flush()
        self.log.flush()

# Перенаправляем stdout в файл и консоль
sys.stdout = DualLogger(report_path)

# Настраиваем logging для записи в тот же файл
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(report_path, mode='a', encoding='utf-8'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

print("=" * 80)
print(f"🚀 ОЦЕНКА КАЧЕСТВА RETRIEVAL - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)
print(f"📁 Корневая директория проекта: {project_root}")
print(f"📝 Отчёт сохраняется в: {report_path}")

🚀 ОЦЕНКА КАЧЕСТВА RETRIEVAL - 2025-11-17 12:02:13
📁 Корневая директория проекта: /srv/nlp1/eval_dir/T-bank_NLP_
📝 Отчёт сохраняется в: /srv/nlp1/eval_dir/T-bank_NLP_/src/eval/report.txt


✅ Импорты выполнены успешно
🔄 Инициализация RetrieverService...
✅ RetrieverService инициализирован
📊 Загружено 4991 запросов
✅ После фильтрации (индексы 0,1,2,5,6,7,10,11,12...): 999 запросов
📈 Это 20.0% от исходного набора

Пример запроса: Запущено ли сообщество для инвесторов в приложении Альфа-Инвестиций только для Android или планируется и для iOS?
Ground truth: channel=1418181070, message=7078
✅ Функция evaluate_query определена
🚀 Начало оценки для 999 запросов
📊 Получаем топ-10 результатов, считаем метрики для K = [1, 3, 5, 10]
2025-11-17 12:03:01,239 - WARNING - Попытка 1/3 не удалась, повторяю...

[100/999] Промежуточные метрики:
  K=1: Hit Rate=0.5400 (54.00%), MRR=0.5400
  K=3: Hit Rate=0.7500 (75.00%), MRR=0.6317
  K=5: Hit Rate=0.7900 (79.00%), MRR=0.6412
  K=10: Hit Rate=0.8500 (85.00%), MRR=0.6493

[200/999] Промежуточные метрики:
  K=1: Hit Rate=0.4900 (49.00%), MRR=0.4900
  K=3: Hit Rate=0.7000 (70.00%), MRR=0.5850
  K=5: Hit Rate=0.7800 (78.00%), MRR=0.6038
  K=10: Hit

In [3]:
from tplexity.retriever.retriever_service import RetrieverService
from tqdm import tqdm

print("✅ Импорты выполнены успешно")

## 2. Инициализация RetrieverService

In [4]:
# Инициализация RetrieverService
print("🔄 Инициализация RetrieverService...")
retriever = RetrieverService()
print("✅ RetrieverService инициализирован")

## 3. Загрузка запросов из queries.json

In [5]:
# Загружаем queries
queries_path = project_root / "src/eval/eval_data/queries.json"

with open(queries_path, "r", encoding="utf-8") as f:
    all_queries = json.load(f)

print(f"📊 Загружено {len(all_queries)} запросов")

# Для тестирования можно взять подмножество
NUM_QUERIES = None  # Установите число для теста, или None для всех запросов
queries_subset = all_queries[:NUM_QUERIES] if NUM_QUERIES else all_queries

# Фильтруем запросы: берем только индексы 0,1,2,5,6,7,10,11,12,...
# Паттерн: берем 3 элемента, пропускаем 2 (условие: индекс % 5 < 3)
queries = [q for i, q in enumerate(queries_subset) if i % 5 < 1]

print(f"✅ После фильтрации (индексы 0,1,2,5,6,7,10,11,12...): {len(queries)} запросов")
print(f"📈 Это {len(queries)/len(queries_subset)*100:.1f}% от исходного набора")
print(f"\nПример запроса: {queries[0]['query']}")
print(f"Ground truth: channel={queries[0]['id_channel']}, message={queries[0]['id_message']}")

## 4. Функция оценки для одного запроса

In [6]:
async def evaluate_query(query_data: dict, retriever, top_k: int = 10, k_values: list = [1, 3, 5, 10], max_retries: int = 3) -> dict:
    """
    Оценивает качество поиска для одного запроса с retry механизмом
    
    Args:
        query_data: Данные запроса из queries.json
        retriever: RetrieverService
        top_k: Количество документов для получения (должно быть >= max(k_values))
        k_values: Список значений K для расчета метрик
        max_retries: Максимальное количество попыток при ошибке
    
    Returns:
        dict: Результаты оценки для всех K
    """
    query_text = query_data["query"]
    ground_truth_id = f"{query_data['id_channel']}_{query_data['id_message']}"
    
    # Выполняем поиск топ-K документов с retry
    search_results = None
    last_error = None
    
    for attempt in range(max_retries):
        try:
            search_results = await retriever.search(
                query=query_text,
                top_k=top_k,
                top_n=top_k,
                use_rerank=False
            )
            break  # Успешно - выходим из цикла retry
        except Exception as e:
            last_error = e
            if attempt < max_retries - 1:
                logger.warning(f"Попытка {attempt+1}/{max_retries} не удалась, повторяю...")
                await asyncio.sleep(1)  # Небольшая задержка перед retry
            else:
                logger.error(f"Ошибка при поиске после {max_retries} попыток: {e}")
    
    # Если после всех попыток результата нет
    if search_results is None:
        # Возвращаем нулевые метрики
        error_result = {
            "query": query_text,
            "ground_truth_id": ground_truth_id,
            "gt_position": -1,
            "error": str(last_error) if last_error else "Unknown error",
            "top_k_ids": []
        }
        for k in k_values:
            error_result[f"hit_rate@{k}"] = 0.0
            error_result[f"mrr@{k}"] = 0.0
            error_result[f"pool_recall@{k}"] = 0.0
        return error_result
    
    if not search_results or len(search_results) == 0:
        # Пустые результаты - нулевые метрики
        empty_result = {
            "query": query_text,
            "ground_truth_id": ground_truth_id,
            "gt_position": -1,
            "top_k_ids": []
        }
        for k in k_values:
            empty_result[f"hit_rate@{k}"] = 0.0
            empty_result[f"mrr@{k}"] = 0.0
            empty_result[f"pool_recall@{k}"] = 0.0
        return empty_result
    
    # Находим позицию ground truth документа в результатах
    gt_position = -1  # -1 означает "не найден"
    for pos, (doc_id, score, text, metadata) in enumerate(search_results, start=1):
        doc_id_from_metadata = metadata.get("doc_id", "") if metadata else ""
        if doc_id_from_metadata == ground_truth_id:
            gt_position = pos
            break
    
    # Формируем результат
    result = {
        "query": query_text,
        "ground_truth_id": ground_truth_id,
        "gt_position": gt_position,
        "top_k_ids": [metadata.get("doc_id", "") if metadata else "" for _, _, _, metadata in search_results]
    }
    
    # Вычисляем метрики для каждого K
    for k in k_values:
        # 1. Hit Rate@K (Recall@K) - нашли ли якорный документ в топ-K
        if gt_position > 0 and gt_position <= k:
            hit_rate = 1.0
        else:
            hit_rate = 0.0
        result[f"hit_rate@{k}"] = hit_rate
        
        # 2. MRR@K - обратный ранг (если документ в топ-K)
        if gt_position > 0 and gt_position <= k:
            mrr = 1.0 / gt_position
        else:
            mrr = 0.0
        result[f"mrr@{k}"] = mrr
        
        # 3. Pool-normalized Recall@K
        # У нас всегда 1 релевантный документ (якорный)
        # Pool = топ-10 результатов поиска
        # Формула: (количество релевантных в топ-K) / min(всего релевантных в пуле, K)
        
        # Релевантных в пуле (топ-10): 1 если документ найден, 0 если нет
        relevant_in_pool = 1 if gt_position > 0 else 0
        
        # Релевантных в топ-K: 1 если документ в топ-K, 0 иначе
        relevant_in_topk = 1 if (gt_position > 0 and gt_position <= k) else 0
        
        # Нормализация
        if relevant_in_pool > 0:
            # min(1, K) = 1 для любого K >= 1
            pool_recall = relevant_in_topk / min(relevant_in_pool, k)
        else:
            pool_recall = 0.0
        
        result[f"pool_recall@{k}"] = pool_recall
    
    return result

print("✅ Функция evaluate_query определена")


## 5. Запуск оценки для всех запросов

In [7]:
# Конфигурация
TOP_K = 10  # Получаем топ-10 результатов
K_VALUES = [1, 3, 5, 10]  # Считаем метрики для этих значений K

print(f"🚀 Начало оценки для {len(queries)} запросов")
print(f"📊 Получаем топ-{TOP_K} результатов, считаем метрики для K = {K_VALUES}")
print("=" * 80)

# Словари для хранения метрик по каждому K
all_results = []
metrics_by_k = {k: {"hit_rates": [], "mrrs": [], "pool_recalls": []} for k in K_VALUES}

# Обрабатываем каждый запрос
for i, query_data in enumerate(tqdm(queries, desc="Оценка запросов")):
    result = await evaluate_query(query_data, retriever, top_k=TOP_K, k_values=K_VALUES)
        
    all_results.append(result)
    
    # Собираем метрики для каждого K
    for k in K_VALUES:
        metrics_by_k[k]["hit_rates"].append(result[f"hit_rate@{k}"])
        metrics_by_k[k]["mrrs"].append(result[f"mrr@{k}"])
        metrics_by_k[k]["pool_recalls"].append(result[f"pool_recall@{k}"])
    
    # Логируем каждый 100-й результат
    if (i + 1) % 100 == 0:
        print(f"\n[{i+1}/{len(queries)}] Промежуточные метрики:")
        for k in K_VALUES:
            current_hit_rate = sum(metrics_by_k[k]["hit_rates"]) / len(metrics_by_k[k]["hit_rates"])
            current_mrr = sum(metrics_by_k[k]["mrrs"]) / len(metrics_by_k[k]["mrrs"])
            print(f"  K={k}: Hit Rate={current_hit_rate:.4f} ({current_hit_rate*100:.2f}%), MRR={current_mrr:.4f}")

print("\n" + "=" * 80)
print("✅ Оценка завершена!")

Оценка запросов: 100%|██████████| 999/999 [04:33<00:00,  3.65it/s] 


## 6. Вычисление итоговых метрик

In [8]:
# Вычисляем агрегированные метрики для каждого K
final_metrics = {}
for k in K_VALUES:
    hit_rate = sum(metrics_by_k[k]["hit_rates"]) / len(metrics_by_k[k]["hit_rates"])
    mrr = sum(metrics_by_k[k]["mrrs"]) / len(metrics_by_k[k]["mrrs"])
    pool_recall = sum(metrics_by_k[k]["pool_recalls"]) / len(metrics_by_k[k]["pool_recalls"])
    
    final_metrics[k] = {
        "hit_rate": hit_rate,
        "mrr": mrr,
        "pool_recall": pool_recall,
        "num_hits": sum(metrics_by_k[k]["hit_rates"])  # Количество запросов, где документ найден
    }

# Выводим результаты
print("=" * 80)
print("📊 ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("=" * 80)
print(f"Количество запросов: {len(queries)}")
print(f"Получено топ-{TOP_K} результатов для каждого запроса")
print(f"Значения K для оценки: {K_VALUES}")
print("\n" + "=" * 80)

# Таблица метрик
print("\n🎯 МЕТРИКИ ПО КАЖДОМУ K:\n")
print(f"{'K':>4} | {'Hit Rate@K':>12} | {'MRR@K':>8} | {'Pool Recall@K':>15} | {'Найдено':>10}")
print("-" * 70)
for k in K_VALUES:
    m = final_metrics[k]
    print(f"{k:>4} | {m['hit_rate']:>11.4f} | {m['mrr']:>8.4f} | {m['pool_recall']:>15.4f} | {int(m['num_hits']):>4}/{len(queries)}")

print("\n" + "=" * 80)
print("\n📝 Интерпретация метрик:")
print("  • Hit Rate@K (Recall@K): доля запросов, где якорный документ попал в топ-K")
print("  • MRR@K: среднее значение 1/позиция для документов в топ-K (выше = лучше)")
print("  • Pool Recall@K: доля релевантных документов в топ-K (нормированная)")
print("=" * 80)

## 7. Детальный анализ результатов

In [9]:
# Статистика по найденным документам
from collections import Counter

# Общая статистика по топ-10
found_in_top10 = sum(1 for r in all_results if r.get("gt_position", -1) > 0)
not_found = len(queries) - found_in_top10

print(f"\n📈 Детальная статистика (топ-{TOP_K}):")
print(f"  Якорный документ найден в топ-{TOP_K}: {found_in_top10}/{len(queries)} ({found_in_top10/len(queries)*100:.1f}%)")
print(f"  Якорный документ НЕ найден: {not_found}/{len(queries)} ({not_found/len(queries)*100:.1f}%)")

# Распределение позиций GT документа
positions = [r["gt_position"] for r in all_results if r["gt_position"] > 0]

if positions:
    distribution = Counter(positions)
    print(f"\n📊 Распределение позиций якорного документа (когда найден):")
    for pos in sorted(distribution.keys()):
        freq = distribution[pos]
        print(f"  Позиция {pos}: {freq} запросов ({freq/found_in_top10*100:.1f}% от найденных)")
    
    avg_position = sum(positions) / len(positions)
    median_position = sorted(positions)[len(positions)//2]
    print(f"\n  📍 Средняя позиция: {avg_position:.2f}")
    print(f"  📍 Медианная позиция: {median_position}")

# Детальная статистика по каждому K
print(f"\n" + "=" * 80)
print("📊 ДЕТАЛЬНАЯ СТАТИСТИКА ПО K:")
print("=" * 80)
for k in K_VALUES:
    found_in_k = int(final_metrics[k]["num_hits"])
    not_in_k = len(queries) - found_in_k
    print(f"\n🔹 K = {k}:")
    print(f"   Найдено: {found_in_k} ({found_in_k/len(queries)*100:.1f}%)")
    print(f"   Не найдено: {not_in_k} ({not_in_k/len(queries)*100:.1f}%)")
    print(f"   Hit Rate@{k}: {final_metrics[k]['hit_rate']:.4f}")
    print(f"   MRR@{k}: {final_metrics[k]['mrr']:.4f}")

## 8. Примеры результатов

In [10]:
# Показываем примеры успешных и неудачных запросов
print("\n" + "=" * 80)
print("📋 ПРИМЕРЫ РЕЗУЛЬТАТОВ")
print("=" * 80)

# Успешные и неудачные примеры
successful = [r for r in all_results if r["gt_position"] > 0]
failed = [r for r in all_results if r["gt_position"] == -1]

print(f"\n✅ Примеры УСПЕШНЫХ запросов (якорный документ найден в топ-{TOP_K}):")
print("-" * 80)
for i, result in enumerate(successful[:3], 1):
    print(f"\n[Пример {i}]")
    print(f"Query: {result['query'][:100]}...")
    print(f"Якорный doc_id: {result['ground_truth_id']}")
    print(f"Позиция в результатах: {result['gt_position']}")
    print(f"Метрики: Hit@1={result['hit_rate@1']:.0f}, Hit@3={result['hit_rate@3']:.0f}, Hit@5={result['hit_rate@5']:.0f}, Hit@10={result['hit_rate@10']:.0f}")
    print(f"MRR@10: {result['mrr@10']:.4f}")
    print(f"Топ-5 найденных doc_ids: {result['top_k_ids'][:5]}")

print(f"\n\n❌ Примеры НЕУДАЧНЫХ запросов (якорный документ НЕ найден в топ-{TOP_K}):")
print("-" * 80)
for i, result in enumerate(failed[:3], 1):
    print(f"\n[Пример {i}]")
    print(f"Query: {result['query'][:100]}...")
    print(f"Якорный doc_id: {result['ground_truth_id']}")
    print(f"Топ-5 найденных doc_ids: {result['top_k_ids'][:5]}")

## 9. Сохранение результатов

In [ ]:
# Сохраняем результаты в JSON
# Подготавливаем словарь метрик
metrics_dict = {}
for k in K_VALUES:
    metrics_dict[f"hit_rate@{k}"] = final_metrics[k]["hit_rate"]
    metrics_dict[f"mrr@{k}"] = final_metrics[k]["mrr"]
    metrics_dict[f"pool_recall@{k}"] = final_metrics[k]["pool_recall"]
    metrics_dict[f"num_hits@{k}"] = int(final_metrics[k]["num_hits"])

output_data = {
    "config": {
        "num_queries": len(queries),
        "top_k": TOP_K,
        "k_values": K_VALUES,
        "use_rerank": False,
        "filter_pattern": "indices 0,1,2,5,6,7,... (i % 5 < 3)",
        "timestamp": datetime.now().isoformat()
    },
    "metrics": metrics_dict,
    "summary": {
        "total_queries": len(queries),
        "found_in_top10": found_in_top10,
        "not_found": not_found,
        "avg_position": avg_position if positions else None,
        "median_position": median_position if positions else None
    },
    "detailed_results": all_results
}

output_path = project_root / "src/eval/eval_results_multi_k.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Результаты сохранены в: {output_path}")
print(f"\n📊 Сводка итоговых метрик:")
for k in K_VALUES:
    print(f"  K={k}: Hit Rate={final_metrics[k]['hit_rate']:.4f}, MRR={final_metrics[k]['mrr']:.4f}, Pool Recall={final_metrics[k]['pool_recall']:.4f}")

print("\n" + "=" * 80)
print(f"📝 ПОЛНЫЙ ОТЧЁТ СОХРАНЁН В: {report_path}")
print(f"📋 JSON РЕЗУЛЬТАТЫ: {output_path}")
print("=" * 80)

# Закрываем файл логирования
if hasattr(sys.stdout, 'log'):
    sys.stdout.log.close()
    print("✅ Отчёт успешно сохранён!")

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/ipykernel/utils.py", line 60, in run_in_context
    return await asyncio.create_task(coro, context=context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 614, in shell_main
    await self.dispatch_shell(msg, subshell_id=subshell_id)
  File "/srv/nlp1/eval_dir/T-bank_NLP_/.venv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 484, in dispatch_shell
    sys.stdout.flush()
  File "/tmp/ipykernel_2012067/1879823494.py", line 29, in flush
    self.log.flush()
ValueError: I/O operation on closed file.
